# Multi-Layer Perceptron

In [2]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
from torchvision import datasets, transforms
from torchvision.io import read_image
from torch import nn, optim
from matplotlib import cm
from matplotlib import style
%matplotlib widget
style.use('default')
#https://www.pluralsight.com/guides/building-your-first-pytorch-solution

## Data normalization

In [3]:
img_size = 224

#transform = transforms.Compose([transforms.ToTensor(),
#                                transforms.Resize((img_size, img_size)),
#                                transforms.Normalize((0.5,), (0.5,))])

#No normalization as document says.
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Resize((img_size, img_size))])

In [4]:
#Code just to ease the load from local folder
import os
print(os.listdir("."))

from os import walk
for (dirpath, dirnames, filenames) in walk("."):
    print("Directory path: ", dirpath)
    print("Folder name: ", dirnames)
#     print("File name: ", filenames)

['dataset_train_test', 'dataset_train_test.zip', 'PAPILA_dataset.pdf', 'Readme.txt', 'Tests.docx', 'TP1 - Newton-Rhapson.ipynb', 'TP1 - Optimizacion de Funciones.ipynb', 'TP1 - Perceptron multi-capa.ipynb', 'TP_1_Optimizacion_Redes_Neuronales.lyx', 'TP_1_Optimizacion_Redes_Neuronales.lyx~', 'TP_1_Optimizacion_Redes_Neuronales.pdf']
Directory path:  .
Folder name:  ['dataset_train_test']
Directory path:  .\dataset_train_test
Folder name:  ['dataset_train_test']
Directory path:  .\dataset_train_test\dataset_train_test
Folder name:  ['test', 'train']
Directory path:  .\dataset_train_test\dataset_train_test\test
Folder name:  ['0', '1']
Directory path:  .\dataset_train_test\dataset_train_test\test\0
Folder name:  []
Directory path:  .\dataset_train_test\dataset_train_test\test\1
Folder name:  []
Directory path:  .\dataset_train_test\dataset_train_test\train
Folder name:  ['0', '1']
Directory path:  .\dataset_train_test\dataset_train_test\train\0
Folder name:  []
Directory path:  .\dataset_

## Loading data

torch.Size([32, 3, 224, 224])


C:\Users\jcord\AppData\Local\Temp\ipykernel_5660\3510772727.py:31: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matricesor `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2985.)
  axs[0,0].imshow(images[0].T.numpy().squeeze(), cmap='gray_r')


In [5]:
%matplotlib widget
#def load_test_train_data_MNIST():
#    trainset = datasets.MNIST(r'..\input\MNIST', download=True, train=True, transform=transform)
#    trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

#    testset = datasets.MNIST(r'..\input\MNIST', download=True, train=False, transform=transform)
#    testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)
#    return trainloader, testloader

from locale import normalize

from matplotlib.pyplot import imshow


def load_test_train_data_tp1():
    trainset = torchvision.datasets.ImageFolder(r'.\dataset_train_test\dataset_train_test\train',transform = transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)

    testset = torchvision.datasets.ImageFolder(r'.\dataset_train_test\dataset_train_test\test',transform = transform)
    testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=True)
    return trainloader, testloader

trainloader, testloader = load_test_train_data_tp1()
dataiter = iter(trainloader) # creating a iterator
images, labels = dataiter.next() # creating images for image and lables for image number (0 to 9) 

print(images.mT.shape)
#plt.imshow(images[0].T.numpy().squeeze(), cmap='gray_r');

fig, axs = plt.subplots(2,2)
axs[0,0].imshow(images[0].T.numpy().squeeze(), cmap='gray_r')
axs[0,1].imshow(images[1].T.numpy().squeeze(), cmap='gray_r')
axs[1,0].imshow(images[2].T.numpy().squeeze(), cmap='gray_r')
axs[1,1].imshow(images[3].T.numpy().squeeze(), cmap='gray_r')


## Create Multi-layer perceptron network model

In [6]:
def create_MLP_model():
    # Model creation with neural net Sequential model
    model=nn.Sequential(nn.Linear(224*224*3,128), # 1 layer:- 784 input 128 o/p
                        #nn.Tanh(),
                        nn.ReLU(),          # Defining Regular linear unit as activation
                        #nn.Sigmoid(),
                        nn.Linear(128,64),  # 2 Layer:- 128 Input and 64 O/p
                        #nn.Tanh(),          # Defining Regular linear unit as activation
                        nn.Sigmoid(),
                        nn.ReLU(),
                        nn.Linear(64,2),   # 3 Layer:- 64 Input and 10 O/P as (0-9)
                        nn.LogSoftmax(dim=1) # Defining the log softmax to find the probablities for the last output unit
                      ) 
    return model


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Running on device: ", device)
mlp_model = create_MLP_model()

#moving models to device
mlp_model.to(device)
#create error criterion
criterion = nn.NLLLoss() 
print("MLP model")
print(mlp_model)


Running on device:  cuda
MLP model
Sequential(
  (0): Linear(in_features=150528, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=64, bias=True)
  (3): Sigmoid()
  (4): ReLU()
  (5): Linear(in_features=64, out_features=2, bias=True)
  (6): LogSoftmax(dim=1)
)


## Train model

In [7]:
def train_model(model, criterion, epochs = 15, lr = 0.01):

    time0 = time()    
    running_loss_list= []
    epochs_list = []
    optimizer = optim.SGD(model.parameters(), lr= lr, momentum=0.9)
    for e in range(epochs):
        running_loss = 0
        
        #go for every batch
        for images, labels in trainloader:
            #move data to specific device
            images = images.to(device)
            labels = labels.to(device)
            # Flatenning MNIST images with size [64,784] (for MLP)
            
            images = images.view(images.shape[0], -1) 
            
            # defining gradient in each epoch as 0
            optimizer.zero_grad()            
            # modeling for each image batch
            output = model(images)

            # calculating the loss
            loss = criterion(output, labels)

            # This is where the model learns by backpropagating
            loss.backward()

            # And optimizes its weights here
            optimizer.step()

            # calculating the loss
            running_loss += loss.item()

        else:
            print("Epoch {} - Training loss: {}".format(e, running_loss/len(trainloader)))
    print("\nTraining Time (in minutes) =",(time()-time0)/60)
    return model

print("Training MLP model")
print("Device", next(mlp_model.parameters()).is_cuda)
mlp_model = train_model(mlp_model, criterion, epochs = 15, lr = 0.003)

Training MLP model
Device True
Epoch 0 - Training loss: 0.6819527707993984
Epoch 1 - Training loss: 0.6421978138387203
Epoch 2 - Training loss: 0.5862601809203625
Epoch 3 - Training loss: 0.5145760364830494
Epoch 4 - Training loss: 0.4537316747009754
Epoch 5 - Training loss: 0.38848790992051363
Epoch 6 - Training loss: 0.3752443930134177
Epoch 7 - Training loss: 0.38966580014675856
Epoch 8 - Training loss: 0.33304932340979576
Epoch 9 - Training loss: 0.3126353118568659
Epoch 10 - Training loss: 0.3184081148356199
Epoch 11 - Training loss: 0.267120398581028
Epoch 12 - Training loss: 0.31081565376371145
Epoch 13 - Training loss: 0.33574297558516264
Epoch 14 - Training loss: 0.3286671740934253

Training Time (in minutes) = 0.603176991144816


## Test model

In [8]:
def test_model_mlp(testloader, model):
    correct_count, all_count = 0, 0
    for images,labels in testloader:
      #move data to specific device
      images = images.to(device)
      labels = labels.to(device)
      for i in range(len(labels)):
        img = images[i].view(1, 224*224*3)
        #evaluate model with no grad
        with torch.no_grad():
            logps = model(img)
        ps = torch.exp(logps)
        probab = list(ps.cpu().numpy()[0])
        #get predicted label
        pred_label = probab.index(max(probab))
        true_label = labels.cpu().numpy()[i]
        if(true_label == pred_label):
          correct_count += 1
        all_count += 1

    print("Number Of Images Tested =", all_count)
    print("Number Of Correct Tests =", correct_count)
    print("\nModel Accuracy =", (correct_count/all_count))



print("Testing MLP model")
test_model_mlp(testloader, mlp_model)

Testing MLP model
Number Of Images Tested = 210
Number Of Correct Tests = 183

Model Accuracy = 0.8714285714285714


torch.Size([3, 224, 224])


In [29]:
#Punto 3.1 del TP01
#Gaussian blur filter
def gaussian_blur():
    #Gaussian blur filter
    F = torch.tensor([[0.0625,0.125,0.0625],[0.125,0.25,0.125],[0.0625,0.125,0.0625]]).float()
    return F

### Convlution for unsharp masking
def calculate_target_size(img_size: int, kernel_size: int) -> int:
    num_pixels = 0
    
    # From 0 up to img size (if img size = 224, then up to 223)
    for i in range(img_size):
        # Add the kernel size (let's say 3) to the current i
        added = i + kernel_size
        # It must be lower than the image size
        if added <= img_size:
            # Increment if so
            num_pixels += 1
            
    return num_pixels

#Manual convlution method for 3 channel images.
def convolution(U,F):

    new_size = calculate_target_size(img_size=U.shape[1],kernel_size=U.shape[0])

    kernel=F

    kernel_shape = kernel.shape[0]

    conv_img = torch.zeros(3,new_size,new_size)
    
    for g in range(3):
        for i in range(new_size):
            for j in range(new_size):
                mat = U[g,i:i+kernel_shape,j:j+kernel_shape]
                conv_img[g, i, j] = torch.sum(torch.multiply(mat,kernel))

    #Add padding to the convuluted image so the size is the same for original and convoluted image.
    conv_img=torchvision.transforms.Pad(1)(conv_img)
    
    return conv_img

def unsharp_masking(U,amount):
    return U + (U-convolution(U,gaussian_blur())) * amount

U = read_image(r'.\dataset_train_test\dataset_train_test\train\0\Im106_ACRIMA.jpg') / 255
U=torchvision.transforms.Resize(224)(U)
print(U.shape)

def disp(x): return (x.clamp(0,1) * 255).byte().permute(1,2,0).cpu()

fig, (ax, bx, cx, dx) = plt.subplots(1,4, figsize=(10,10),label="Convolution test")
ax.imshow(disp(U))
bx.imshow(disp(unsharp_masking(U,10)))
cx.imshow(disp(unsharp_masking(U,30)))
dx.imshow(disp(unsharp_masking(U,100)))
plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)


In [30]:
def create_MLP_model():
    # Model creation with neural net Sequential model
    model=nn.Sequential(nn.Linear(224*224*3,128), # 1 layer:- 784 input 128 o/p
                        #nn.Tanh(),
                        nn.ReLU(),          # Defining Regular linear unit as activation
                        #nn.Sigmoid(),
                        nn.Linear(128,64),  # 2 Layer:- 128 Input and 64 O/p
                        #nn.Tanh(),          # Defining Regular linear unit as activation
                        nn.Sigmoid(),
                        nn.ReLU(),
                        nn.Linear(64,2),   # 3 Layer:- 64 Input and 10 O/P as (0-9)
                        nn.LogSoftmax(dim=1) # Defining the log softmax to find the probablities for the last output unit
                      ) 
    return model

def create_AlexNet_model():
    class AlexNet(nn.Module):
        def __init__(self):
            super(AlexNet, self).__init__()

            self.layer1 = nn.Sequential(
                nn.Conv2d(224*224*3, 96, kernel_size=11, stride=4, padding=0),
                nn.BatchNorm2d(96),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size = 3, stride = 2))
            self.layer2 = nn.Sequential(
                nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2),
                nn.BatchNorm2d(256),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size = 3, stride = 2))
            self.layer3 = nn.Sequential(
                nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1),
                nn.BatchNorm2d(384),
                nn.ReLU())
            self.layer4 = nn.Sequential(
                nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1),
                nn.BatchNorm2d(384),
                nn.ReLU())
            self.layer5 = nn.Sequential(
                nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
                nn.BatchNorm2d(256),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size = 3, stride = 2))
            self.fc = nn.Sequential(
                nn.Dropout(0.5),
                nn.Linear(9216, 4096),
                nn.ReLU())
            self.fc1 = nn.Sequential(
                nn.Dropout(0.5),
                nn.Linear(4096, 4096),
                nn.ReLU())
            self.fc2= nn.Sequential(
                nn.Linear(4096, 2))

        def forward(self, x):
            out = self.layer1(x)
            out = self.layer2(out)
            out = self.layer3(out)
            out = self.layer4(out)
            out = self.layer5(out)
            out = out.reshape(out.size(0), -1)
            out = self.fc(out)
            out = self.fc1(out)
            out = self.fc2(out)
            return out
 
    cnn = AlexNet()
    return cnn

    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Running on device: ", device)
mlp_model = create_MLP_model()

#moving models to device
mlp_model.to(device)
cnn_model = create_AlexNet_model()
cnn_model.to(device)
#error function
criterion = nn.NLLLoss() 
print("MLP model")
print(mlp_model)
print("CNN model ")
print(cnn_model)

Running on device:  cuda


RuntimeError: CUDA out of memory. Tried to allocate 6.52 GiB (GPU 0; 10.00 GiB total capacity; 503.47 MiB already allocated; 6.52 GiB free; 1.01 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [25]:
def train_model(model, criterion, epochs = 15, lr = 0.01, is_MLP = False):

    time0 = time()    
    running_loss_list= []
    epochs_list = []
    optimizer = optim.SGD(model.parameters(), lr= lr, momentum=0.9)
    for e in range(epochs):
        running_loss = 0
        
        #go for every batch
        for images, labels in trainloader:
            #move data to specific device
            images = images.to(device)
            labels = labels.to(device)
            # Flatenning MNIST images with size [64,784] (for MLP)
            if(is_MLP):
              images = images.view(images.shape[0], -1) 
            
            # defining gradient in each epoch as 0
            optimizer.zero_grad()            
            # modeling for each image batch
            output = model(images)

            # calculating the loss
            loss = criterion(output, labels)

            # This is where the model learns by backpropagating
            loss.backward()

            # And optimizes its weights here
            optimizer.step()

            # calculating the loss
            running_loss += loss.item()

        else:
            print("Epoch {} - Training loss: {}".format(e, running_loss/len(trainloader)))
    print("\nTraining Time (in minutes) =",(time()-time0)/60)
    return model
print("Training CNN model")
cnn_model = train_model(cnn_model, criterion, epochs = 15, lr = 0.01, is_MLP = False)

print("Training MLP model")
mlp_model = train_model(mlp_model, criterion, epochs = 15, lr = 0.01, is_MLP = True)

Training CNN model


RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x6400 and 9216x4096)